In [ ]:
!python -m pip install geopy
!python -m pip install searoute
import pandas as pd
import math
import numpy as np
import datetime
from io import StringIO
import random
from geopy.distance import geodesic
import searoute
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
import datetime
from io import StringIO
import random

rs = 420
random.seed(rs)

print(datetime.datetime.today().strftime("%HH:%MM %dd"))


print(datetime.datetime.today().strftime("%HH:%MM %dd"))
# Load datasets

featuers = ['vesselId', 'hour', 'day', 'month', 'minute', 'vesselType', 'yearBuilt', 'length', 'breadth', 'CEU', 'DWT', 'GT', 'port_latitude', 'port_longitude', 'hour_eta', 'day_eta', 'month_eta', 'minute_eta', 'lat_shift_1', 'lon_shift_1', 'minute_shift_1', 'hour_shift_1', 'day_shift_1', 'month_shift_1', 'lat_shift_2', 'lon_shift_2', 'minute_shift_2', 'hour_shift_2', 'day_shift_2', 'month_shift_2', 'lat_shift_3', 'lon_shift_3', 'minute_shift_3', 'hour_shift_3', 'day_shift_3', 'month_shift_3', 'lat_shift_4', 'lon_shift_4', 'minute_shift_4', 'hour_shift_4', 'day_shift_4', 'month_shift_4', 'cog', 'heading', 'time_diff']
vessel_encoder = LabelEncoder()
timesteps = 5
# test-test split
liste = []
for i in range(len(featuers)):
    liste.append(pd.read_csv('x_data_new'+str(i)+'.csv', sep=','))
liste.append(pd.read_csv('x_data_60.csv', sep=','))

X = liste[0].join(liste[1:])
X['vesselId'] = vessel_encoder.fit_transform(X['vesselId'])
print(X.head(1))
y_1 = pd.read_csv('y_data_1.csv', sep=',')
y_2 = pd.read_csv('y_data_2.csv', sep=',')
y = y_2.join(y_1)

print("port:", y.columns)
# Use the same features as training
X_0 = pd.read_csv('x_test_new.csv', sep=',')
X_1 = pd.read_csv('x_test_new_1.csv', sep=',')
X_2 = pd.read_csv('x_test_new_2.csv', sep=',')
X_3 = pd.read_csv('x_test_new_3.csv', sep=',')
X_4 = pd.read_csv('x_test_new_4.csv', sep=',')
X_test = X_0.join([X_1, X_2, X_3, X_4])

X_test['vesselId'] = vessel_encoder.transform(X_test['vesselId'])
y_test = pd.read_csv('ais_test_predictions_7.csv', sep=',')
print(y_test.head(10))
# Scale the test set

last = X_test.join(y_test)
X_test2 = X_test.join(y_test)
last.rename(columns = {'latitude_predicted':'latitude_searoute', 'longitude_predicted':'longitude_searoute'}, inplace=True)
X_test2.rename(columns = {'latitude_predicted':'latitude_searoute', 'longitude_predicted':'longitude_searoute'}, inplace=True)


print("X_test2", X_test2[['latitude_searoute', 'longitude_searoute', 'lat_shift_1', 'lat_shift_2']].head(10))

for i in range(1, timesteps):  # Only 3 previous time steps
    X_test2[f'lat_shift_{i}'] = X_test2.groupby('vesselId')['latitude_searoute'].shift(i)
    X_test2[f'lon_shift_{i}'] = X_test2.groupby('vesselId')['longitude_searoute'].shift(i)
    X_test2[f'lat_shift_{i}'] = np.where(X_test2[f'lat_shift_{i}'].isna(), last[f'lat_shift_{i}'], X_test2[f'lat_shift_{i}'])
    X_test2[f'lon_shift_{i}'] = np.where(X_test2[f'lon_shift_{i}'].isna(), last[f'lon_shift_{i}'], X_test2[f'lon_shift_{i}'])

print("X_test2", X_test2[['port_latitude', 'port_longitude', 'lat_shift_1', 'lat_shift_2']].head(10))


# Helper functions for distance, bearing, and destination point calculations
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in kilometers
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c  # Distance in km

def calculate_bearing(lat1, lon1, lat2, lon2):
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    delta_lambda = math.radians(lon2 - lon1)

    x = math.sin(delta_lambda) * math.cos(phi2)
    y = math.cos(phi1) * math.sin(phi2) - math.sin(phi1) * math.cos(phi2) * math.cos(delta_lambda)

    bearing = math.atan2(x, y)
    return (math.degrees(bearing) + 360) % 360  # Normalize to 0-360 degrees

def destination_point(lat, lon, distance_km, bearing_deg):
    R = 6371  # Radius of Earth in kilometers
    bearing = math.radians(bearing_deg)
    lat, lon = math.radians(lat), math.radians(lon)

    new_lat = math.asin(math.sin(lat) * math.cos(distance_km / R) +
                        math.cos(lat) * math.sin(distance_km / R) * math.cos(bearing))
    new_lon = lon + math.atan2(math.sin(bearing) * math.sin(distance_km / R) * math.cos(lat),
                               math.cos(distance_km / R) - math.sin(lat) * math.sin(new_lat))

    return math.degrees(new_lat), math.degrees(new_lon)

# Sample DataFrame with necessary columns
# Calculate speed and bearing for each vessel entry
X = X.join(y)

X.rename(columns = {'latitude':'latitude_searoute', 'longitude':'longitude_searoute'}, inplace=True)
print("xcol:", X.columns)

for idx, row in X.iterrows():

    # Retrieve the last known predicted coordinates, if available
    lat1, lon1 = row['lat_shift_1'], row['lon_shift_1']
    lat2, lon2 = row['lat_shift_2'], row['lon_shift_2']
  
    port_lat, port_lon = row['port_latitude'], row['port_longitude']
    
    # Step 1: Use searoute to calculate the distance between the last two known positions
    distance_km = haversine(lat2, lon2, lat1, lon1)  # Distance in kilometers
    timediff_2_hours = row['time_diff_1'] / 3600  # Convert seconds to hours
    speed_kmh = distance_km / timediff_2_hours  # Speed in km/h

    bearing_to_port = calculate_bearing(lat1, lon1, port_lat, port_lon)
    timediff_1_hours = row['time_diff'] / 3600
    distance_to_travel = speed_kmh * timediff_1_hours
    
    origin = [lon1, lat1]
    destination = [port_lon, port_lat]
    #print(origin, destination)
    route = searoute.searoute(origin, destination)  # This returns a GeoJSON LineString
    
    # Extract the coordinates from the GeoJSON route
    route_coords = route['geometry']['coordinates']
    
    # Step 4: Traverse the route coordinates to find the predicted position
    cumulative_distance = 0
    predicted_lat, predicted_lon = lat1, lon1  # Initialize with the starting point

    for i in range(1, len(route_coords)):
        # Calculate the segment distance
        point_a = (route_coords[i-1][1], route_coords[i-1][0])  # (lat, lon)
        point_b = (route_coords[i][1], route_coords[i][0])  # (lat, lon)
        segment_distance = geodesic(point_a, point_b).kilometers

        # Add segment distance to cumulative distance
        cumulative_distance += segment_distance

        # Check if cumulative distance has reached or exceeded the distance to travel
        if cumulative_distance >= distance_to_travel:
            predicted_lat, predicted_lon = point_b
            break
        else:
            predicted_lat, predicted_lon = point_b  # Update to last point reached

    # Update the DataFrame with the predicted coordinates
    X.at[idx, 'longitude_searoute'] = predicted_lon
    X.at[idx, 'latitude_searoute'] = predicted_lat

print('blir ferding med training', X[['longitude_searoute', 'latitude_searoute']])


for idx, row in X_test2.iterrows():

    # Retrieve the last known predicted coordinates, if available

    lat1, lon1 = row['lat_shift_1'], row['lon_shift_1']
    lat2, lon2 = row['lat_shift_2'], row['lon_shift_2']
  
    port_lat, port_lon = row['port_latitude'], row['port_longitude']
    
    # Step 1: Use searoute to calculate the distance between the last two known positions
    distance_km = haversine(lat2, lon2, lat1, lon1)  # Distance in kilometers
    timediff_2_hours = row['time_diff_1'] / 3600  # Convert seconds to hours
    speed_kmh = distance_km / timediff_2_hours  # Speed in km/h

    bearing_to_port = calculate_bearing(lat1, lon1, port_lat, port_lon)
    timediff_1_hours = row['time_diff'] / 3600
    distance_to_travel = speed_kmh * timediff_1_hours
    
    origin = [lon1, lat1]
    destination = [port_lon, port_lat]
    print(origin, destination)
    route = searoute.searoute(origin, destination)  # This returns a GeoJSON LineString
    
    # Extract the coordinates from the GeoJSON route
    route_coords = route['geometry']['coordinates']
    
    # Step 4: Traverse the route coordinates to find the predicted position
    cumulative_distance = 0
    predicted_lat, predicted_lon = lat1, lon1  # Initialize with the starting point

    for i in range(1, len(route_coords)):
        # Calculate the segment distance
        point_a = (route_coords[i-1][1], route_coords[i-1][0])  # (lat, lon)
        point_b = (route_coords[i][1], route_coords[i][0])  # (lat, lon)
        segment_distance = geodesic(point_a, point_b).kilometers

        # Add segment distance to cumulative distance
        cumulative_distance += segment_distance

        # Check if cumulative distance has reached or exceeded the distance to travel
        if cumulative_distance >= distance_to_travel:
            predicted_lat, predicted_lon = point_b
            break
        else:
            predicted_lat, predicted_lon = point_b  # Update to last point reached

    # Update the DataFrame with the predicted coordinates
    X_test2.at[idx, 'longitude_searoute'] = predicted_lon
    X_test2.at[idx, 'latitude_searoute'] = predicted_lat

featuers = ['vesselId', 'hour', 'day', 'month', 'minute', 'vesselType', 'yearBuilt', 'length', 'breadth', 'CEU', 'DWT', 'GT', 'port_latitude', 'port_longitude', 'hour_eta', 'day_eta', 'month_eta', 'minute_eta', 'lat_shift_1', 'lon_shift_1', 'lat_shift_2', 'lon_shift_2', 'lat_shift_3', 'lon_shift_3', 'lat_shift_4', 'lon_shift_4', 'cog', 'heading', 'time_diff', 'time_diff_1', 'latitude_searoute', 'longitude_searoute']

X = X[featuers]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.001, random_state=rs)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

model = ExtraTreesRegressor(n_estimators=182, random_state=rs)

# Train a RandomForest model
model.fit(X_train_scaled, y_train)

last = X_test2[featuers]

#TODO: use predictions_df in the slides for the next two predicts

last_scaled = scaler.transform(last)

final_prediction = model.predict(last_scaled)

predictions_df = pd.DataFrame(final_prediction, columns=['longitude_predicted', 'latitude_predicted'])

predictions_df.to_csv('ais_test_predictions_11.csv', index=True)
    #print(f"Predicted next coordinates for vessel {row['vesselId']}: Latitude: {predicted_lat}, Longitude: {predicted_lon}")